In [11]:
#!/usr/bin/env python3
"""
Auto-generate WRH YouTube thumbnails by dropping as many full words as will fit
from the start of a script into a fixed text box.

Requires: Pillow  (pip install pillow)

Templates:
  0 -> /Users/marcus/Downloads/WRH_white.png
  1 -> /Users/marcus/Downloads/WRH_black.png
"""

from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from pathlib import Path
import textwrap
import os

# ---------- CONFIG YOU ASKED FOR ----------
TEMPLATES = {
    0: "/Users/marcus/Downloads/WRH_white.png",
    1: "/Users/marcus/Downloads/WRH_black.png",
}
OUT_DIR = Path("/Users/marcus/Downloads/video_thumbnails_reddit1")

# Text box: x=50, y=200, width=650, height=450
BOX_X, BOX_Y, BOX_W, BOX_H = 40, 260,1200, 450

# -----------------------------------------

def load_arial_font(font_size: int) -> ImageFont.FreeTypeFont:
    """
    Try common macOS Arial locations; fall back to PIL default if not found.
    """
    candidates = [
        "/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Supplemental/Arial.ttf",
        "/System/Library/Fonts/Arial.ttf",
        "/System/Library/Fonts/Arial Unicode.ttf",
    ]
    for p in candidates:
        if Path(p).exists():
            return ImageFont.truetype(p, font_size)
    # Fallback
    return ImageFont.load_default()

def measure_text_height(lines, font: ImageFont.FreeTypeFont, line_spacing_px: int, draw: ImageDraw.ImageDraw) -> int:
    """Total height for given lines with spacing."""
    heights = []
    for line in lines:
        # textbbox gives (left, top, right, bottom)
        bbox = draw.textbbox((0, 0), line, font=font)
        heights.append(bbox[3] - bbox[1])
    if not heights:
        return 0
    return sum(heights) + line_spacing_px * (len(lines) - 1)

def wrap_to_fit(words, draw, font, max_width, max_height, line_spacing_px):
    """
    Greedily add words while re-wrapping; stop when height would overflow.
    Returns the final list of lines that fit and the number of words used.
    """
    used = 0
    lines = []

    # Keep a growing buffer of words; re-wrap on width each iteration.
    for i in range(1, len(words) + 1):
        candidate_text = " ".join(words[:i])

        # Width wrap using textwrap but verify width with draw.textlength (pixel accurate)
        # Start with an optimistic wrap by average chars-per-line; then fix by measuring.
        # We’ll do a manual greedy width wrap using words to be robust.
        wrapped = []
        current = []
        for w in words[:i]:
            test = (" ".join(current + [w])).strip()
            if draw.textlength(test, font=font) <= max_width:
                current.append(w)
            else:
                # line full → commit and start new line
                if current:
                    wrapped.append(" ".join(current))
                # word longer than line: hard-break the single long word
                if draw.textlength(w, font=font) > max_width:
                    # fallback: chop by characters until it fits
                    piece = ""
                    for ch in w:
                        if draw.textlength((piece + ch), font=font) <= max_width:
                            piece += ch
                        else:
                            if piece:
                                wrapped.append(piece)
                            piece = ch
                    if piece:
                        current = [piece]
                    else:
                        current = []
                else:
                    current = [w]
        if current:
            wrapped.append(" ".join(current))

        # Check height
        h = measure_text_height(wrapped, font, line_spacing_px, draw)
        if h <= max_height:
            lines = wrapped
            used = i
        else:
            break

    return lines, used

def generate_thumbnail(
    template_choice: int,
    script_text: str,
    font_size: int = 72,          # change as you like
    font_color: tuple = (0, 0, 0), # default black text; switch to white for dark template if you want
    line_spacing_px: int = 6
) -> Path:
    # resolve template
    if template_choice not in TEMPLATES:
        raise ValueError("template_choice must be 0 (white) or 1 (black)")
    template_path = Path(TEMPLATES[template_choice])
    if not template_path.exists():
        raise FileNotFoundError(f"Template not found: {template_path}")

    # choose default text color based on template background
    if template_choice == 1 and font_color == (0, 0, 0):
        # black template → default to white text
        font_color = (255, 255, 255)

    img = Image.open(template_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    font = load_arial_font(font_size)

    # Build as many full words as fit
    words = script_text.strip().split()
    lines, used_words = wrap_to_fit(
        words, draw, font,
        max_width=BOX_W,
        max_height=BOX_H,
        line_spacing_px=line_spacing_px
    )

    # Draw the lines
    cursor_y = BOX_Y
    for line in lines:
        draw.text((BOX_X, cursor_y), line, font=font, fill=font_color)
        # advance y by exact line height + spacing
        bbox = draw.textbbox((0, 0), line, font=font)
        line_h = bbox[3] - bbox[1]
        cursor_y += line_h + line_spacing_px

    # Ensure output dir
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # Timestamped name
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = "WRH_white" if template_choice == 0 else "WRH_black"
    out_path = OUT_DIR / f"{base}_{stamp}.png"
    img.save(out_path)

    print(f"Saved thumbnail: {out_path}")
    print(f"Words used: {used_words}/{len(words)}")
    return out_path

# ---------- Example usage ----------
if __name__ == "__main__":
    # Example script text; replace with your content.
    example_script = (
        "Reddit exposed: AITA stories that spiraled out of control — "
        "here’s what really happened when the comments dug deeper."
    )

    # 0 = white template, 1 = black template
    generate_thumbnail(
        template_choice=0,
        script_text=example_script,
        font_size=30,          # <— set your N here
        # font_color=(0,0,0),  # optional override
        line_spacing_px=5
    )


Saved thumbnail: /Users/marcus/Downloads/video_thumbnails_reddit1/WRH_white_20250903_160638.png
Words used: 19/19


In [18]:
#!/usr/bin/env python3
"""
WRH thumbnail generator with boldness/thickness control and robust measurements.

Requires: Pillow  (pip install pillow)
Templates:
  0 -> /Users/marcus/Downloads/WRH_white.png
  1 -> /Users/marcus/Downloads/WRH_black.png
"""

from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from pathlib import Path
import os

# ---------- CONFIG ----------
TEMPLATES = {
    0: "/Users/marcus/Downloads/WRH_white.png",
    1: "/Users/marcus/Downloads/WRH_black.png",
}
OUT_DIR = Path("/Users/marcus/Downloads/video_thumbnails_reddit1")

# Text box: x=40, y=260, width=1200, height=450
BOX_X, BOX_Y, BOX_W, BOX_H = 40, 260, 1200, 450
# ---------------------------

def load_arial_font(font_size: int, weight: str = "regular") -> ImageFont.FreeTypeFont:
    weight = (weight or "regular").lower()
    if weight not in {"regular", "bold"}:
        weight = "regular"

    if weight == "bold":
        candidates = [
            "/Library/Fonts/Arial Bold.ttf",
            "/System/Library/Fonts/Supplemental/Arial Bold.ttf",
            "/System/Library/Fonts/Arial Bold.ttf",
        ]
    else:
        candidates = [
            "/Library/Fonts/Arial.ttf",
            "/System/Library/Fonts/Supplemental/Arial.ttf",
            "/System/Library/Fonts/Arial.ttf",
            "/System/Library/Fonts/Arial Unicode.ttf",
        ]

    for p in candidates:
        if Path(p).exists():
            return ImageFont.truetype(p, font_size)
    return ImageFont.load_default()

# ---------- measurement helpers (handle older Pillow gracefully) ----------
def _bbox(draw: ImageDraw.ImageDraw, text: str, font, thickness_px: int):
    """Get text bounding box; fall back if stroke_width unsupported."""
    try:
        return draw.textbbox((0, 0), text, font=font, stroke_width=thickness_px)
    except TypeError:
        # Older Pillow: measure without stroke; pad a bit so we don't overflow
        b = draw.textbbox((0, 0), text, font=font)
        # pad width by ~2*thickness and height by ~thickness to be conservative
        return (b[0], b[1], b[2] + 2 * thickness_px, b[3] + thickness_px)

def _text_width(draw, text, font, thickness_px: int) -> int:
    b = _bbox(draw, text, font, thickness_px)
    return b[2] - b[0]

def _line_height(draw, text, font, thickness_px: int) -> int:
    b = _bbox(draw, text, font, thickness_px)
    return b[3] - b[1]
# -------------------------------------------------------------------------

def measure_text_height(lines, font: ImageFont.FreeTypeFont,
                        line_spacing_px: int, draw: ImageDraw.ImageDraw,
                        thickness_px: int = 0) -> int:
    heights = [_line_height(draw, line, font, thickness_px) for line in lines]
    if not heights:
        return 0
    return sum(heights) + line_spacing_px * (len(lines) - 1)

def wrap_to_fit(words, draw, font, max_width, max_height, line_spacing_px, thickness_px: int):
    used = 0
    lines = []

    for i in range(1, len(words) + 1):
        wrapped, current = [], []
        for w in words[:i]:
            test = (" ".join(current + [w])).strip()
            if _text_width(draw, test, font, thickness_px) <= max_width:
                current.append(w)
            else:
                if current:
                    wrapped.append(" ".join(current))
                # If a single word is wider than the line, hard-break it by chars
                if _text_width(draw, w, font, thickness_px) > max_width:
                    piece = ""
                    for ch in w:
                        if _text_width(draw, piece + ch, font, thickness_px) <= max_width:
                            piece += ch
                        else:
                            if piece:
                                wrapped.append(piece)
                            piece = ch
                    current = [piece] if piece else []
                else:
                    current = [w]
        if current:
            wrapped.append(" ".join(current))

        h = measure_text_height(wrapped, font, line_spacing_px, draw, thickness_px)
        if h <= max_height:
            lines = wrapped
            used = i
        else:
            break

    return lines, used

def generate_thumbnail(
    template_choice: int,
    script_text: str,
    font_size: int = 72,                 # size N
    font_color: tuple = (0, 0, 0),       # default black text
    line_spacing_px: int = 6,
    font_weight: str = "regular",        # "regular" or "bold"
    thickness_px: int = 0                # extra thickness via same-color stroke (0..3+)
) -> Path:
    if template_choice not in TEMPLATES:
        raise ValueError("template_choice must be 0 (white) or 1 (black)")
    template_path = Path(TEMPLATES[template_choice])
    if not template_path.exists():
        raise FileNotFoundError(f"Template not found: {template_path}")

    if template_choice == 1 and font_color == (0, 0, 0):
        font_color = (255, 255, 255)

    img = Image.open(template_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    font = load_arial_font(font_size, weight=font_weight)

    words = script_text.strip().split()
    lines, used_words = wrap_to_fit(
        words, draw, font,
        max_width=BOX_W,
        max_height=BOX_H,
        line_spacing_px=line_spacing_px,
        thickness_px=thickness_px
    )

    cursor_y = BOX_Y
    for line in lines:
        # draw with stroke if available; fallback silently if not
        try:
            draw.text(
                (BOX_X, cursor_y),
                line,
                font=font,
                fill=font_color,
                stroke_width=thickness_px,
                stroke_fill=font_color
            )
        except TypeError:
            draw.text((BOX_X, cursor_y), line, font=font, fill=font_color)

        line_h = _line_height(draw, line, font, thickness_px)
        cursor_y += line_h + line_spacing_px

    OUT_DIR.mkdir(parents=True, exist_ok=True)
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = "WRH_white" if template_choice == 0 else "WRH_black"
    out_path = OUT_DIR / f"{base}_{stamp}.png"
    img.save(out_path)

    print(f"Saved thumbnail: {out_path}")
    print(f"Words used: {used_words}/{len(words)}")
    return out_path

# ---------- Example usage ----------
if __name__ == "__main__":
    example_script = (
        "Reddit exposed: AITA stories that spiraled out of control — "
        "here’s what really happened when the comments dug deeper."
    )

    generate_thumbnail(
        template_choice=1,           # 0=white, 1=black
        script_text=example_script,
        font_size=50,
        line_spacing_px=5,
        font_weight="bold",          # "regular" or "bold"
        thickness_px=0.5            # 0..3+ to increase thickness
    )


Saved thumbnail: /Users/marcus/Downloads/video_thumbnails_reddit1/WRH_black_20250903_160923.png
Words used: 19/19


In [2]:
#!/usr/bin/env python3
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from pathlib import Path

# ---------- CONFIG ----------
TEMPLATES = {
    0: "/Users/marcus/Downloads/WRH_white.png",
    1: "/Users/marcus/Downloads/WRH_black.png",
}
OUT_DIR = Path("/Users/marcus/Downloads/video_thumbnails_reddit1")

# Text box: x=40, y=260, width=1200, height=450
BOX_X, BOX_Y, BOX_W, BOX_H = 40, 260, 1200, 360
# ---------------------------

def load_arial_font(font_size: int, weight: str = "regular"):
    weight = (weight or "regular").lower()
    if weight == "bold":
        candidates = [
            "/Library/Fonts/Arial Bold.ttf",
            "/System/Library/Fonts/Supplemental/Arial Bold.ttf",
            "/System/Library/Fonts/Arial Bold.ttf",
        ]
    else:
        candidates = [
            "/Library/Fonts/Arial.ttf",
            "/System/Library/Fonts/Supplemental/Arial.ttf",
            "/System/Library/Fonts/Arial.ttf",
            "/System/Library/Fonts/Arial Unicode.ttf",
        ]
    for p in candidates:
        if Path(p).exists():
            return ImageFont.truetype(p, font_size)
    return ImageFont.load_default()

# ---- robust measurements (support older Pillow too) ----
def _bbox(draw, text, font, thickness_px: int):
    try:
        return draw.textbbox((0, 0), text, font=font, stroke_width=thickness_px)
    except TypeError:
        b = draw.textbbox((0, 0), text, font=font)
        return (b[0], b[1], b[2] + 2*thickness_px, b[3] + thickness_px)

def _text_width(draw, text, font, thickness_px: int) -> int:
    b = _bbox(draw, text, font, thickness_px)
    return b[2] - b[0]

def _line_height(draw, text, font, thickness_px: int) -> int:
    b = _bbox(draw, text, font, thickness_px)
    return b[3] - b[1]
# -------------------------------------------------------

def measure_text_height(lines, font, line_spacing_px, draw, thickness_px: int = 0) -> int:
    heights = [_line_height(draw, line, font, thickness_px) for line in lines]
    return (sum(heights) + line_spacing_px * (len(lines) - 1)) if heights else 0

def wrap_to_fit(words, draw, font, max_width, max_height, line_spacing_px, thickness_px: int):
    """Greedily add full words while re-wrapping; stop right before height overflows."""
    used = 0
    lines = []
    for i in range(1, len(words) + 1):
        wrapped, current = [], []
        for w in words[:i]:
            test = (" ".join(current + [w])).strip()
            if _text_width(draw, test, font, thickness_px) <= max_width:
                current.append(w)
            else:
                if current:
                    wrapped.append(" ".join(current))
                # if a single word is wider than the line, char-split it
                if _text_width(draw, w, font, thickness_px) > max_width:
                    piece = ""
                    for ch in w:
                        if _text_width(draw, piece + ch, font, thickness_px) <= max_width:
                            piece += ch
                        else:
                            if piece:
                                wrapped.append(piece)
                            piece = ch
                    current = [piece] if piece else []
                else:
                    current = [w]
        if current:
            wrapped.append(" ".join(current))

        h = measure_text_height(wrapped, font, line_spacing_px, draw, thickness_px)
        if h <= max_height:
            lines = wrapped
            used = i
        else:
            break
    return lines, used

def _apply_ellipsis(lines, draw, font, max_width, thickness_px: int, ellipsis="…"):
    """Append ellipsis to last line, trimming by words (then chars) so it fits."""
    if not lines:
        return lines
    last = lines[-1]
    # quick fit
    if _text_width(draw, last + ellipsis, font, thickness_px) <= max_width:
        lines[-1] = last + ellipsis
        return lines

    # word-level backoff
    tokens = last.split(" ")
    while len(tokens) > 1:
        tokens.pop()  # drop last word
        candidate = " ".join(tokens)
        if _text_width(draw, candidate + ellipsis, font, thickness_px) <= max_width:
            lines[-1] = candidate + ellipsis
            return lines

    # single long word: char-level trim
    base = tokens[0] if tokens else last
    trimmed = ""
    for ch in base:
        if _text_width(draw, trimmed + ch + ellipsis, font, thickness_px) <= max_width:
            trimmed += ch
        else:
            break
    if trimmed:
        lines[-1] = trimmed + ellipsis
    else:
        # fallback: ellipsis alone if it fits
        if _text_width(draw, ellipsis, font, thickness_px) <= max_width:
            lines[-1] = ellipsis
        # else leave as-is (shouldn’t happen with reasonable sizes)
    return lines

def generate_thumbnail(
    template_choice: int,
    script_text: str,
    font_size: int = 72,
    font_color: tuple = (0, 0, 0),
    line_spacing_px: int = 6,
    font_weight: str = "regular",    # "regular" | "bold"
    thickness_px: int = 0,           # extra thickness via same-color stroke
    use_ellipsis: bool = True
) -> Path:
    if template_choice not in TEMPLATES:
        raise ValueError("template_choice must be 0 (white) or 1 (black)")
    template_path = Path(TEMPLATES[template_choice])
    if not template_path.exists():
        raise FileNotFoundError(f"Template not found: {template_path}")

    if template_choice == 1 and font_color == (0, 0, 0):
        font_color = (255, 255, 255)

    img = Image.open(template_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    font = load_arial_font(font_size, weight=font_weight)

    words = script_text.strip().split()
    lines, used_words = wrap_to_fit(
        words, draw, font,
        max_width=BOX_W,
        max_height=BOX_H,
        line_spacing_px=line_spacing_px,
        thickness_px=thickness_px
    )

    # If we didn't consume all words, smart-crop and add ellipsis to the last line
    if use_ellipsis and used_words < len(words) and lines:
        lines = _apply_ellipsis(lines, draw, font, BOX_W, thickness_px)

    # draw lines
    cursor_y = BOX_Y
    for line in lines:
        try:
            draw.text((BOX_X, cursor_y), line, font=font,
                      fill=font_color, stroke_width=thickness_px, stroke_fill=font_color)
        except TypeError:  # very old Pillow: no stroke args
            draw.text((BOX_X, cursor_y), line, font=font, fill=font_color)
        cursor_y += _line_height(draw, line, font, thickness_px) + line_spacing_px

    OUT_DIR.mkdir(parents=True, exist_ok=True)
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = "WRH_white" if template_choice == 0 else "WRH_black"
    out_path = OUT_DIR / f"{base}_{stamp}.png"
    img.save(out_path)
    print(f"Saved thumbnail: {out_path}")
    print(f"Words used: {used_words}/{len(words)}  (ellipsis added: {use_ellipsis and used_words < len(words)})")
    return out_path

# --- Example ---
if __name__ == "__main__":
    example_script = (''' 
    Throwaway because my IRL circle knows my main, and this is the kind of thing you don’t get to unsay once it’s out there. Ages for context: me 36F, ex-husband 38M (let’s call him "Mark"), former friend 36F ("Lena"). And no, I don’t need legal advice; the divorce papers are signed and collecting dust in a folder I can’t quite bring myself to shred. What I need, apparently, is to figure out when the floor disappeared from under me — and whether I’m the only one who heard the thud. I am... exhausted. Like, bone-deep, burnt-toast exhausted. If there were a trophy for being the default adult, I’d have it in three sizes and dusted weekly: I paid the bills, I took care of his mother when she got sick, I planned birthdays and remember every allergy and appointment and sent cards to his coworkers when they had babies. I did what needed to be done. And, stupidly, I kept thinking: that’s what love is, right? The sweat and the lists and the load-bearing beams no one notices unless they fail.

    
    
    ''')
    generate_thumbnail(
        template_choice=1,
        script_text=example_script,
        font_size=46,
        line_spacing_px=5,
        font_weight="bold",
        thickness_px=0.5,
        use_ellipsis=True
    )


Saved thumbnail: /Users/marcus/Downloads/video_thumbnails_reddit1/WRH_black_20250903_163951.png
Words used: 67/181  (ellipsis added: True)
